In [2]:
from torch.utils.data import Dataset, DataLoader
import cv2 as cv
from torchvision import transforms, utils
from torchvision.io import read_image
from torchvision.transforms import functional as TF
import torch
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from tqdm.notebook import tqdm    
import json
from typing import List, Dict, Tuple

# CACD2000

In [4]:
data_path = "/mnt/d/chinghsien/Downloads/NTU_Thesis_Code_backup/Datasets/CACD2000/"
j_file = open("/mnt/d/chinghsien/Documents/NTU/Master_Thesis/AGMixer/annotations/CACD2000_aligned.json", "r")
json_afad = json.load(j_file)

pool = [0, 1, 2, 3, 4, 5]
name_list = []
age_list = []

for idx, i in enumerate(json_afad):
    c_name = i['name'].replace("_", " ")
    name_list.append(c_name)
    age_list.append(i["age"])
 

print(min(age_list))
print(max(age_list))

14
62


In [5]:
print(len(name_list))
print(name_list[3])
name_list = list(set(name_list))
print(len(name_list))
print(name_list[3])

163446
Robin Williams
2000
Jon Stewart


# Search Gender from wikipedia

In [5]:
import requests
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

# The Code base is from chatGPT

wiki_url = "https://en.wikipedia.org/wiki/"

def get_gender_from_text(text):
    text_lower = text.lower()
    if " she " in text_lower:
        return "Female"
    elif " he " in text_lower:
        return "Male"
    return "Not found"


def get_gender_from_wikipedia(name):
    try:
        url = wiki_url + name.replace(" ", "_")
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        paragraphs = soup.find_all('p')
        for paragraph in paragraphs:
            text = paragraph.get_text()
            gender = get_gender_from_text(text)
            if gender != "Not found":
                return name, gender
        return name, "Not found"
    except Exception as e:
        return name, f"Error: {e}"

def fetch_genders(names):
    results = []
    with ThreadPoolExecutor(max_workers=10) as executor:
        future_to_name = {executor.submit(get_gender_from_wikipedia, name): name for name in names}
        with tqdm(total=len(future_to_name)) as pbar:
            for future in as_completed(future_to_name):
                name = future_to_name[future]
                try:
                    result = future.result()
                    results.append(result)
                except Exception as e:
                    results.append((name, f"Error: {e}"))
                pbar.update(1)
    return results

results = fetch_genders(name_list)



100%|██████████| 2000/2000 [03:38<00:00,  9.14it/s]


In [6]:
a = get_gender_from_wikipedia("Katie Findlay")
print(a)

('Katie Findlay', 'Not found')


In [7]:
# 輸出結果

gender_dict = {}
for name, gender in results:
    gender_dict[name] = gender
    
print(gender_dict)

{'Agnes Bruckner': 'Female', 'Mary McDonnell': 'Female', 'James Denton': 'Male', 'Minnie Driver': 'Female', 'Seth Green': 'Male', 'Cuba Gooding Jr.': 'Male', 'AnnaLynne McCord': 'Female', 'Cristine Rose': 'Female', 'Chris Evans': 'Not found', 'Heather Thomas': 'Female', 'Colin Farrell': 'Male', 'Michael Moore': 'Male', 'Bryce Dallas Howard': 'Female', 'Deborah Kara Unger': 'Female', 'Taylor Schilling': 'Female', 'Greg Evigan': 'Male', 'James Frain': 'Male', 'Paul Feig': 'Male', 'Bruce Willis': 'Male', 'Thomas Dekker': 'Not found', 'Keith David': 'Male', 'Ian Harding': 'Male', 'Vera Farmiga': 'Female', 'Ashley Williams': 'Not found', 'Sophia Bush': 'Female', 'James Read': 'Male', 'Jeff Garlin': 'Male', 'Stephen Merchant': 'Male', 'Julia Stiles': 'Female', 'Denis Villeneuve': 'Male', 'Gwyneth Paltrow': 'Female', 'Eddie Redmayne': 'Male', 'Renee Olstead': 'Female', 'Nina Dobrev': 'Female', 'Lady Gaga': 'Female', 'Brant Daugherty': 'Male', 'Jack Huston': 'Male', 'John Turturro': 'Male', 'S

In [8]:
name_not_found_list = []
for name, gender in results:
    if gender != "Male" and gender != "Female":
        # print(f"Name: {name}, Gender: {gender}")
        name_not_found_list.append(name)
        
print(len(name_not_found_list))
        
j_file = open("/mnt/d/chinghsien/Downloads/NTU_Thesis_Code_backup/Datasets/A_call/Facial-Age-Estimation-Benchmark-Databases/IMDB-EM-CNN.json", "r")
json_imdb = json.load(j_file)


name_imdb_list = []
gender_imdb_dict = {}
for idx, i in enumerate(json_imdb):
    cname = i["name"]
    name_imdb_list.append(cname.lower())
    gender_imdb_dict[cname.lower()] = i["gender"]

name_imdb_list = list(set(name_imdb_list))

name_imdb_found_list = []
name_not_found_list2 = []
for name in name_not_found_list:
    if name.lower() in name_imdb_list:
        # print(f"name: {name}, gender: {gender_imdb_dict[name.lower()]}")
        if gender_imdb_dict[name.lower()] == "M":
            gender_dict[name] = "Male"
        elif gender_imdb_dict[name.lower()] == "F":
            gender_dict[name] = "Female"
    else:
        name_not_found_list2.append(name)
        
for key in gender_dict:
    gender = gender_dict[key]
    if gender != "Female" and gender != "Male":
        print(key)

print(name_not_found_list2)

59
Ryan Murphy
Aaron Johnson
Charles Baker
Scott Gordon-Patterson
Philip Davis
Todd Bosley
['Ryan Murphy', 'Aaron Johnson', 'Charles Baker', 'Scott Gordon-Patterson', 'Philip Davis', 'Todd Bosley']


# Handle Not Found Manually

In [9]:
gender_dict["Aaron Johnson"] = 'Male'
gender_dict["Philip Davis"] = 'Male'
gender_dict["Todd Bosley"] = 'Male'
gender_dict["Scott Gordon-Patterson"] = 'Male'
gender_dict["Charles Baker"] = 'Male'
gender_dict["Ryan Murphy"] = 'Male'

# Check if still Not found 

for key in gender_dict:
    gender = gender_dict[key]
    if gender != "Female" and gender != "Male":
        print(key)


# Update Gender to JSON

In [10]:
new_json_afad = []
for idx, data in enumerate(json_afad):
    name = data['name'].replace("_", " ")
    gender = "M" if gender_dict[name] == "Male" else "F"
    gender_pair = {"gender": gender}
    data.update(gender_pair)
    new_json_afad.append(data)
        

with open("/mnt/d/chinghsien/Downloads/NTU_Thesis_Code_backup/Datasets/A_call/Facial-Age-Estimation-Benchmark-Databases/CACD2000_aligned_update_gender.json", 'w') as f:
    json.dump(new_json_afad, f, indent=4)

print(new_json_afad[0])
        

{'name': 'Robin_Williams', 'img_path': 'CACD2000/CACD2000/53_Robin_Williams_0001.jpg', 'id_num': 1, 'age': 53, 'birth': 1951, 'lfw': 1, 'rank': 1, 'year': 2004, 'database': 'CACD', 'folder': 1, 'aligned_bbox': [-2, 53, 182, 1, 234, 185, 49, 237], 'alignment_source': 'in the wild', 'gender': 'M'}
